In [2]:
import torch

depth = torch.load('/home/azhuravl/scratch/datasets_latents/driving_1000/002/videos/ref_depths.pt')
depth.shape

/tmp/ipykernel_3382711/4235522030.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  depth = torch.load('/home/azhuravl/scratch/datasets_latents/driving_1000/002/videos/ref

torch.Size([10, 1, 384, 672])

In [3]:
import sys
sys.path.append('/home/azhuravl/work/TrajectoryCrafter/notebooks/22_10_25_scaling_up')
from generate_sceneflow import apply_colormap_to_depth


/CT/video_4d_recon/nobackup/conda_envs/trajcrafter/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/CT/video_4d_recon/nobackup/conda_envs/trajcrafter/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")


In [4]:
sys.path.append('/home/azhuravl/work/TrajectoryCrafter/notebooks/05_11_25_training/lora_utils_ours')

from dataset_videos import SimpleValidationDataset

dataset = SimpleValidationDataset(
    '/home/azhuravl/scratch/datasets_latents/driving_1000',
    max_samples=5, use_depth=True
    )

Found 5 validation samples (depth mode)


In [15]:
data_0 = dataset[0]
# data_0

In [8]:
data_0['warped_video'].shape

torch.Size([3, 49, 384, 672])

In [43]:
def convert_depth_sample_to_rgb(depth_tensor):
    # sample (1, C, T, H, W)
    # input [T, H, W]
    # output [T, H, W, 3]
    
    depth_thw = depth_tensor.mean(dim=1, keepdim=False)[0]  # [T, H, W]
    
    # unnormalize depth to min 1, max 100, 
    # set values where depth_thw is zero to zero
    depth_thw_unnorm = depth_thw * (100.0 - 1.0) + 1.0
    depth_thw_unnorm = torch.where(depth_thw > 0, depth_thw_unnorm, torch.zeros_like(depth_thw_unnorm))
    
    depth_colormap = apply_colormap_to_depth(depth_thw_unnorm, inverse=True)  # [T, H, W, 3]
    depth_bcthw = depth_colormap.permute(3, 0, 1, 2).unsqueeze(0)  # [1, 3, T, H, W]
    return depth_bcthw


In [47]:
depth_bcthw = convert_depth_sample_to_rgb(
    data_0['warped_video'].unsqueeze(0)
    )

In [48]:
depth_bcthw.shape

torch.Size([1, 3, 49, 384, 672])

In [49]:
from videox_fun.utils.utils import save_videos_grid

save_videos_grid(
    depth_bcthw, 
    '/home/azhuravl/work/TrajectoryCrafter/notebooks/05_11_25_training/test_depth_vis.mp4',
)